In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import pandas as pd
import lxml
import re
import numpy as np

# 1. Introduction

# 2. Getting Top 20 List

The purpose of this section is to aggregate the list of players who have been on every year's HLTV Top 20. The entire process is not computationally laborious, so the resulting DataFrame was not saved locally.

The function **_getTop20(year, index)_** accepts a certain year as well as the dictionary of indices to each list's webpage on hltv.com. A couple of exceptions had to be made because it appeared that HLTV changed their HTML webpages between the years 2017 and 2018. This resulted in the player extraction for years 2018 and 2019 to be harder. The function returns an array that has the player names in order from #1 to #20.

The function **_aggregateTop20()_** serves as the main function that uses the helper function **_getTop20_**. I went to each year's introduction page and took down the index numbers, which were unique. Luckily, the rest of the url were the same across all four, making iteration easy. Since **_getTop20_** returns a 20x1 numpy array, the **_aggregateTop20_** makes a 20x4 numpy array. This numpy array was then converted and returned as a DataFrame.

In [2]:
def getTop20(year, index):
    url = 'https://www.hltv.org/news/' + str(index) + '/top-20-players-of-' + str(year) + '-introduction'
    src = requests.get(url).text
    soup = BeautifulSoup(src,'lxml')
    arr = np.array([])
    
    if year in [2016, 2017]:
        for i in soup.find_all('tr'):
            arr = np.concatenate((arr,[i.text.split()[2][1:-1]]))
    elif year in [2018, 2019]:
        top20 = soup.find_all('blockquote')[1].text.strip()
        top20list = re.compile("[0-9]+\.  ").split(top20)
        for player in top20list:
            if not len(player) <= 1:
                arr = np.concatenate((arr,[player.split('"')[1]]))
    else:
        print('Error')
    return arr


def aggregateTop20():  
    top20indices = {2016: 19558, 2017: 22348, 2018: 25735, 2019: 28749}

    top20 = np.array([])
    for year in top20indices:
        top20 = np.concatenate((top20,getTop20(year, top20indices[year]))) 
        time.sleep(1)
    df = pd.DataFrame(top20.reshape(4,20).swapaxes(0,1), columns=list(top20indices.keys()))
    df.index = np.arange(1,21)
    return df

top20DF = aggregateTop20()

# 3. Data Scraping: Feature Extraction

In this section, data scraping begins with finding individual statistics for all the players in a given year. These features will also be saved locally. Later, filtering will reduce the amount of players used in the machine learning model. The purpose of this section is to just save **all** players locally.

## 3-1. Unfiltered Players

Firstly, the player names from each year must be collected from HLTV's repo. Since HLTV did not publish advanced statistics until late 2015, only the years 2016, 2017, 2018, 2019 are used. 

Only LAN matches are counted, while no filter restriction is placed on the Player Filter. The **_oneYear(year)_** function is a helper function that returns a string that has the date range depending on the year supplied. Each year has its own HTML file, and the collection of all four HTML files are placed in the 'unfilteredplayers' folder.

In [3]:
def oneYear(year):
    return 'startDate=' + str(year) + '-01-01&endDate=' + str(year) + '-12-31'

for year in np.arange(2016,2020):
    url = 'https://www.hltv.org/stats/players?' + oneYear(year) + '&matchType=Lan'
    temp = requests.get(url).text
    with open('unfilteredplayers/players' + str(year) + '.html', 'w', encoding='utf-8') as f:
        f.write(temp)
    time.sleep(1)

These html files now must be read and turned into .csv files. The **_allplayersonLAN(year)_** function below takes in a year and reads that year's HTML page that has the list of players who have official statistics on LAN events only. A dictionary is returned that maps players' names to players' individual statistic page based on the year.

In [4]:
def allplayersonLAN(year):
    with open('unfilteredplayers/players' + str(year) + '.html','r',encoding='utf-8') as players_html:
        soup = BeautifulSoup(players_html,'lxml')
    temp = {}
    for player in soup.find_all('td',class_='playerCol'):
        temp[player.text] = 'https://hltv.org' + player.find('a')['href']
    return temp

The **_htmlToCsvUnfiltered(year)_** function is the main function that transforms the HTML files into CSV files. It uses **_allplayersonLAN_** as a helper function to produce a dictionary, which is then converted to a DataFrame and then exported as a .csv. These .csv files are then placed in 'unfilteredplayers' as well, accompanying the .html files.

In [5]:
def htmlToCsvUnfiltered(year): 
    playerlist = allplayersonLAN(year)
    tab = pd.DataFrame.from_dict(playerlist,orient='index',columns=['Webpage'])
    tab.to_csv('unfilteredplayers/players' + str(year) + '.csv')
    print(str(year) + "-Number of unfiltered players: " + str(len(playerlist)))
    return

for year in np.arange(2016,2020):
    htmlToCsvUnfiltered(year)

2016-Number of unfiltered players: 190
2017-Number of unfiltered players: 284
2018-Number of unfiltered players: 252
2019-Number of unfiltered players: 234


## 3-2. Did a player make HLTV?

In Section 2, **_aggregateTop20()_** produced a DataFrame of all the Top 20 lists for the four years. 

In [14]:
top20DF

,2016,2017,2018,2019
1,coldzera,coldzera,s1mple,ZywOo
2,FalleN,NiKo,device,s1mple
3,device,fer,NiKo,device
4,s1mple,rain,electronic,EliGE
5,Snax,device,dupreeh,Magisk
6,shox,FalleN,NAF,electronic
7,f0rest,kennyS,Magisk,NAF
8,olofmeister,s1mple,gla1ve,Brehze
9,ScreaM,GuardiaN,KRIMZ,Twistzz
10,flusha,dupreeh,coldzera,ropz


**_checkIfTop20(row,top20)_** is a function that is to be applied across a DataFrame of all the players. It takes in a DataFrame row as well as a Series that contains the Top 20 based on the provided year. An example is shown below with the players from 2019.

In [7]:
def checkIfTop20(row,top20):
    name = row.name
    if top20.str.contains(name,regex=False).sum() > 0:
        return True
    else:    
        return False

df = pd.read_csv('unfilteredplayers/players2019.csv',index_col=0)
df.insert(df.shape[1],'HLTV Top 20',df.apply(checkIfTop20,axis=1,args=(top20DF.loc[:,2019],)))
df

,Webpage,HLTV Top 20
Jamppi,https://hltv.org/stats/players/14087/Jamppi?st...,False
ZywOo,https://hltv.org/stats/players/11893/ZywOo?sta...,True
s1mple,https://hltv.org/stats/players/7998/s1mple?sta...,True
witz,https://hltv.org/stats/players/13995/witz?star...,False
Papichulo,https://hltv.org/stats/players/15440/Papichulo...,False
...,...,...
daps,https://hltv.org/stats/players/8521/daps?start...,False
tiziaN,https://hltv.org/stats/players/5796/tiziaN?sta...,False
HUNDEN,https://hltv.org/stats/players/7415/HUNDEN?sta...,False
advent,https://hltv.org/stats/players/8600/advent?sta...,False


## 3-3. Features

Feature extraction is based on the three tabs of a player: Overview, Individual, and Matches. A total of 12 statistics will be used as features:

- Rating (0)
- ADR (1)
- KPR (2)
- DPR (3)
- APR (4)
- IMPACT (5)
- KAST (6)
- Grenade dmg (7)
- k-d diff (8)
- Opening kill ratio (9)
- team win percent after 1st kill (10)
- % of maps with 1+ rating (11)

**_getScaledRating(sp)_** is a helper function that scales player rating based on the player's rating against top 5, 10, 20, 30, and 50 opponents. **_getFromOverview(url,arr)_** is the main function that fills in a stats array with the statistics that can be taken from the player's overview page. 

- Rating (0)
- ADR (1)
- KPR (2)
- DPR (3)
- APR (4)
- IMPACT (5)
- KAST (6)
- Grenade dmg (7)

In [8]:
def getScaledRating(sp):
    ratingsTab = sp.find('div',class_='featured-ratings-container')
    scales = {'vs top 5 opponents': 0.4, 'vs top 10 opponents': 0.3, 'vs top 20 opponents': 0.2, 
              'vs top 30 opponents': 0.075, 'vs top 50 opponents': 0.025}
    scaledrating = 0
    for rating in ratingsTab.find_all('div',class_='rating-breakdown'):
        val = 0
        ratingtype = rating.find('div',class_='rating-description').text
        mapcount = int(rating.find('div',class_='rating-maps').text[1:-1].split()[0])
        if mapcount == 0:
            continue 
        
        temp = float(rating.find('div',class_='rating-value').text)
        val = temp * scales[ratingtype]
        
        if ratingtype == 'vs top 5 opponents' and mapcount < 10:
            val *= 0.75
        elif ratingtype== 'vs top 10 opponents' and mapcount < 20:
            val *= 0.75
        elif ratingtype== 'vs top 20 opponents' and mapcount < 40:
            val *= 0.75       
        scaledrating += val

    return scaledrating


def getFromOverview(url, arr):
    src = requests.get(url).text
    soup = BeautifulSoup(src,'lxml')
    statsIndex = {'Rating': 0, 'ADR': 1, 'KPR': 2, 'DPR': 3, 
                  'Assists / round': 4, 'Impact': 5, 'KAST': 6, 'Grenade dmg / Round': 7}
    
    # ADR, KPR, DPR, KAST, IMPACT
    for i in soup.find_all('div',class_=re.compile('summaryStatBreakdown ')):
        statname = i.find('div',class_='summaryStatBreakdownSubHeader').text.split()[0]
        if statname in statsIndex:
            if statname == 'KAST':
                arr[statsIndex[statname]] = i.find('div',class_='summaryStatBreakdownDataValue').text[:-1]
            else:
                arr[statsIndex[statname]] = i.find('div',class_='summaryStatBreakdownDataValue').text
    
    # APR, Grenade Dmg/ Round
    for i in soup.find_all('div',class_='stats-row'):
        if i.text.find('Grenade dmg / Round') > -1:
            arr[statsIndex['Grenade dmg / Round']] = i.find_all('span')[1].text
        elif i.text.find('Assists / round') > -1:
            arr[statsIndex['Assists / round']] = i.find_all('span')[1].text
            
    # Rating Scale
    arr[statsIndex['Rating']] = getScaledRating(soup)
    return arr

**_getFromIndividual(url,arr)_** is the main function that fills in a stats array with the statistics that can be taken from the player's Individual page. 

- k-d diff (8)
- Opening kill ratio (9)
- team win percent after 1st kill (10)

In [9]:
def getFromIndividual(url, arr):
    src = requests.get(url).text
    soup = BeautifulSoup(src,'lxml')
    statsIndex = {'Kill - Death difference': 8, 'Opening kill ratio': 9, 'Team win percent after first kill': 10}
    for i in soup.find_all('div',class_='stats-row'):
        statname = i.span.text
        if statname in statsIndex:
            if statname == 'Kill - Death difference':
                arr[statsIndex[statname]] = i.span.next_sibling.next_sibling.text
            elif statname == 'Team win percent after first kill':
                arr[statsIndex[statname]] = i.span.next_sibling.text[:-1]
            else:
                arr[statsIndex[statname]] = i.span.next_sibling.text
                
    return arr

**_getFromMatches(url,arr)_** is the main function that fills in a stats array with the statistics that can be taken from the player's Matches page. 

- % of maps with 1+ rating (11)

In [10]:
def getFromMatches(url, arr):
    src = requests.get(url).text
    soup = BeautifulSoup(src,'lxml')
    statsIndex = {'Maps with 1+ rating': 11}
    for i in soup.find_all('div',class_='col'):
        statname = i.find('div',class_='description').text
        if statname is None:
            continue
        elif statname in statsIndex:
            arr[statsIndex[statname]] = i.find('div',class_='value').text[:-1]
            break
    return arr

**_getFeatures(row)_** is applied across an entire DataFrame and uses the three functions defined above. It takes in a row and returns a numpy array that contains the statistics. 

In [11]:
def getFeatures(row):
    statsarr = np.zeros((12,))
    
    overvUrl = row['Webpage']
    statsarr = getFromOverview(overvUrl,statsarr)
    time.sleep(7.5)
    indivUrl = overvUrl[:31] + 'individual/' + overvUrl[31:]
    statsarr = getFromIndividual(indivUrl,statsarr)
    time.sleep(7.5)
    matchUrl = overvUrl[:31] + 'matches/' + overvUrl[31:]
    statsarr = getFromMatches(matchUrl,statsarr)
    time.sleep(7.5)
    return statsarr

Since applying **_getFeatures_** returns a Series of 12x1 numpy arrays, this nested structure needs to be flattened into a non-nested two-dimensional array. **_flattenFeatures(s,df)** extracts the nested structure made from the applied function and combines it with the existing DataFrame that came from the end of section 3-1. The Series is first converted into a numpy array, then numpy's **stack** function will flatten it. It is then concatenated with the original DataFrame that just had player names and hltv URLs.

In [12]:
def flattenFeatures(s,df):
    flat = np.stack(s.to_numpy())
   
    featureNames = np.array(['Adj. Rating','ADR','KPR','DPR','APR',
                         'Impact','KAST','NadePR','K-D Diff', 
                         'Opening Kill Ratio', 'Team win % after 1st kill', '% of maps with 1+ rating'])
    features = pd.DataFrame(data=flat, columns=featureNames)
    features.index = df.index
    
    final = pd.concat([df,features],axis=1)
    return final

Since applying **_extractFinalFeature_** is the main function for feature extraction. It takes in a year as well as the top20DF created in section 2. It reads in the DataFrame saved in section 3-1 and then adds a column checking whether or not a player made it in that year's HLTV top 20. It will then call **_getFeatures_** to produce the Series of numpy arrays that contains all the stats. That series is inserted along with the DataFrame into **_flattenFeatures_** to produce a combined DataFrame with all the features necessary for analysis. 

In [16]:
def extractFinalFeatures(year):
    top20DF = aggregateTop20()
    df = pd.read_csv('unfilteredplayers/players' + str(year) + '.csv',index_col=0)
    df.insert(df.shape[1],'HLTV Top 20',df.apply(checkIfTop20,axis=1,args=(top20DF.loc[:,year],)))
    
    statsonly = df.apply(getFeatures,axis=1)
    
    finalTab = flattenFeatures(statsonly,df)
    
    finalTab.to_csv('features/features'+str(year)+'.csv') 
    
    return

extractFinalFeatures(2017)

# 4. Data Scraping: Filtering Players

In [15]:
pd.read_csv('unfilteredplayers/players2016.csv',index_col=0)

,Webpage
XANTARES,https://hltv.org/stats/players/7938/XANTARES?s...
ZELIN,https://hltv.org/stats/players/9001/ZELIN?star...
coldzera,https://hltv.org/stats/players/9216/coldzera?s...
oskar,https://hltv.org/stats/players/798/oskar?start...
device,https://hltv.org/stats/players/7592/device?sta...
...,...
B1ad3,https://hltv.org/stats/players/472/B1ad3?start...
MSL,https://hltv.org/stats/players/7156/MSL?startD...
nkl,https://hltv.org/stats/players/5327/nkl?startD...
freakazoid,https://hltv.org/stats/players/7808/freakazoid...


## Filtering

The **_allplayersonLAN(year)_** function below takes in a year and reads that year's HTML page that has the list of players who have official statistics on LAN events only. A dictionary is returned that maps players' names to players' individual statistic page based on the year.

In [4]:
def allplayersonLAN(year):
    with open('unfilteredplayers/players' + str(year) + '.html','r',encoding='utf-8') as players_html:
        soup = BeautifulSoup(players_html,'lxml')
    temp = {}
    for player in soup.find_all('td',class_='playerCol'):
        temp[player.text] = 'https://hltv.org' + player.find('a')['href']
    return temp

The **_againstTop50(row)_** function checks if each player in the dictionary above has at least played 20 maps against rank top 50 opponents. It takes in a DataFrame row, and this function will be applied across the entire DataFrame using panda's **apply()** function. The return is boolean: true if player has played at least 20 maps against top 50 opponents, and false if player has not. 

This portion of the filtering process might be subject to change. 20 maps against top 50 opponents appears to be a very low bar to clear, might adjust later.

In [28]:
def againstTop50(row):
    name = row.name
    url = row.Webpage
    src = requests.get(url).text
    soup = BeautifulSoup(src,'lxml')
    for i in soup.find_all('div',class_='col-custom'):
        if i.text.find('top 50 opponents') > -1:
            mapsplayed = i.find('div',class_='rating-maps').text.split()[0].split('(')[1]
            if int(mapsplayed) >= 20:
                return True
    time.sleep(7.5)
    return False

This function combines the two above to filter out players that may have not participated in enough important events. A dictionary is formed using allplayersonLAN() and then converted into a DataFrame. The againstTop50 function is applied across every row of the DataFrame to produce a Series that says if each player has or has not played 20 maps against top 50 opponenets.

- 2016-Number of unfiltered players: 190
- 2016-Number of filtered players: 171
- 2017-Number of unfiltered players: 284
- 2017-Number of filtered players: 181
- 2018-Number of unfiltered players: 252
- 2018-Number of filtered players: 223
- 2019-Number of unfiltered players: 234
- 2019-Number of filtered players: 186

In [48]:
def exportFilteredPlayers(year): 
    playerlist = allplayersonLAN(year)
    tab = pd.DataFrame.from_dict(playerlist,orient='index',columns=['Webpage'])
#    tab.insert(tab.shape[1],'AgainstTop50',tab.apply(againstTop50,axis=1))
    tab.to_csv('filteredplayers/players' + str(year) + '.csv')
    print(str(year) + "-Number of unfiltered players: " + str(len(playerlist)))
  #  print(str(year) + "-Number of filtered players: " + str(tab[tab.AgainstTop50].shape[0]))
    return

#for year in np.arange(2016,2020):
exportFilteredPlayers(2016)

2016-Number of unfiltered players: 190
